The image was taken from this [reference](https://medium.com/heuritech/attention-mechanism-5aba9a2d4727).

![image](workshop/attention_tutorial.webp)

![image](workshop/functions.png)

<div class="alert alert-block alert-info">
<b>Example:</b> <br>
- I have a batch which contains 64 sentences <br>
- Each sentence contains 30 words <br>
- Each word is embedded into a vecotor with length 100 => data shape is: (64, 30, 100) <br>
- I pass this input to a Bidirectional LSTM with 128 units <br>
- My goal is to predict whether this is a positive sentence (0) or a negative sentence (1) => Binary classification <br>
</div>

In [8]:
from keras import backend as K
import tensorflow as tf
from keras import Model
from keras.models import Sequential
from keras.layers import Layer, Dense, LSTM, Input, Bidirectional, Embedding

In [3]:
print("tensorflow version:", tf.__version__)

tensorflow version: 2.6.0


In [11]:
input = tf.random.normal([64, 30, 100])
lstm = Bidirectional(LSTM(128, return_sequences=True))
# lstm = LSTM(128, return_sequences=False)

lstm_output = lstm(input)

print("Input shape:", input.shape)
print("Output shape", lstm_output.shape)

Input shape: (64, 30, 100)
Output shape (64, 30, 256)


<div class="alert alert-block alert-info">
<b>Choices:</b> <br>
Feed this output to a Dense layer with 1 neuron to get 0 or 1 for classes, or use attention mechanism.
<br>
</div>

<div class="alert alert-block alert-success">
<b>Steps:</b>
    <ol>
     <li>Calculate the score function (dot product)</li>
     <li>Normalize the scores</li>
     <li>Get the attention weights</li>
     <li>Multiply Values with the attention weights</li>
     <li>Compute the context vector (summation)</li>  
    </ol>
</div>

<div class="alert alert-block alert-warning">
<b>dot product:</b> <br>
    - $\ a = [1,2,3]$ <br>
    - $\ b = [4,5,6]$ <br>
    - $\ a.b = (a[0]*b[0]) + (a[1]*b[1]) + (a[2]*b[2])$ <br>
</div>

<div class="alert alert-block alert-info">
Compute the score function
</div>

In [12]:
attention_input = lstm_output
num_neurons = 1
num_dim_perword = attention_input.shape[-1]
w = tf.random.normal(shape=(num_dim_perword, num_neurons))
score = K.dot(attention_input, w)
print("weight size:", w.shape)
print("attention layer's input size:", attention_input.shape)
print("score shape", score.shape)

weight size: (256, 1)
attention layer's input size: (64, 30, 256)
score shape (64, 30, 1)


<div class="alert alert-block alert-info">
    Let's add a bias and apply an activation function
</div>

In [13]:
num_words_per_sentence = attention_input.shape[-2]
b = tf.random.normal(shape=(num_words_per_sentence, num_neurons))
score = score + b
# apply tanh
score = K.tanh(score)
print("bias shape:", b.shape)
print("score shape", score.shape)

bias shape: (30, 1)
score shape (64, 30, 1)


<div class="alert alert-block alert-warning">
<b>Why would I want to apply tanh?</b>
</div>

In [6]:
score[0]

<tf.Tensor: shape=(30, 1), dtype=float32, numpy=
array([[-0.24817254],
       [-0.9982055 ],
       [-0.98057956],
       [-0.977513  ],
       [-0.9636431 ],
       [-0.99999285],
       [-0.74562144],
       [-0.815487  ],
       [ 0.6806417 ],
       [ 0.6527308 ],
       [ 0.7553818 ],
       [-0.95033544],
       [ 0.5429102 ],
       [ 0.99938333],
       [ 0.05523834],
       [ 0.9359733 ],
       [ 0.9814705 ],
       [ 0.99454844],
       [ 0.9992686 ],
       [-0.7720548 ],
       [ 0.80491376],
       [ 0.99999726],
       [ 0.9896996 ],
       [ 0.9995863 ],
       [ 0.9989417 ],
       [ 0.83792263],
       [ 0.7989326 ],
       [-0.63476455],
       [ 0.14484772],
       [-0.36548555]], dtype=float32)>

<div class="alert alert-block alert-info">
    Get the attention weights by normalizing the weights (Ex: apply softmax)
</div>

In [18]:
attention_weights = K.softmax(K.squeeze(score, axis=-1))
# attention_weights = K.softmax(score)
attention_weights = K.expand_dims(attention_weights, axis=-1)
print("attention_weights shape:", attention_weights.shape)

attention_weights shape: (64, 30, 1)


In [19]:
attention_weights[0]

<tf.Tensor: shape=(30, 1), dtype=float32, numpy=
array([[0.04111318],
       [0.01902021],
       [0.06929121],
       [0.0643251 ],
       [0.05613615],
       [0.00957971],
       [0.01023932],
       [0.05810092],
       [0.04007023],
       [0.00962106],
       [0.040673  ],
       [0.06840713],
       [0.04832774],
       [0.01032592],
       [0.01433442],
       [0.01455746],
       [0.06780814],
       [0.00963962],
       [0.0695413 ],
       [0.0388581 ],
       [0.01073956],
       [0.01159854],
       [0.01002639],
       [0.00990984],
       [0.0101979 ],
       [0.01805307],
       [0.00957428],
       [0.02126205],
       [0.06933418],
       [0.0693344 ]], dtype=float32)>

<div class="alert alert-block alert-info">
Compute the weighted representation of the values
</div>

 ### TODO

In [9]:
print("attention_input shape:", attention_input.shape)
print("attention_weights shape:", attention_weights.shape)
values_weighted_representation = attention_input*attention_weights # here we are using attention input as the values
print("Weighted representation of values (keys in this case):", values_weighted_representation.shape)

attention_input shape: (64, 30, 256)
attention_weights shape: (64, 30, 1)
Weighted representation of values (keys in this case): (64, 30, 256)


<div class="alert alert-block alert-info">
Compute the context vector
</div>

In [10]:
context_vector = K.sum(values_weighted_representation, axis=1)
print("context vector shape:", context_vector.shape)

context vector shape: (64, 256)


<div class="alert alert-block alert-success">
<b>Let's write it in a Layer format that can be applied to a model.</b>
</div>

In [36]:
class Attention(Layer): 
    def __init__(self, num_neurons=1):    
        self.num_neurons = num_neurons
        super(Attention, self).__init__()
        
    def build(self, input_shape):
        self.num_dim_perword = input_shape[-1]
        self.words_pertweet = input_shape[-2]

        self.W = self.add_weight(
            name="att_weight",
            shape=(self.num_dim_perword, self.num_neurons),
            initializer='normal')

        self.b = self.add_weight(
            name="att_bias",
            shape=(self.words_pertweet, self.num_neurons),
            initializer='zero')
        super(Attention, self).build(input_shape)
        
    def call(self, x):
        e = K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        a = K.softmax(e)
        a = K.expand_dims(a,axis=-1)
        output = x*a
        return K.sum(output, axis=1)

<div class="alert alert-block alert-success">
<b>Simple Attention mechanism explained by ChatGPT</b>
</div>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
"""
This implementation uses the TensorFlow Layer class to define the attention layer. 
The build method initializes the weights of the layer (W, b, and u), and the call 
method applies the attention mechanism to the input. The attention mechanism uses 
a tanh activation and a softmax activation to calculate the attention scores, and 
then multiplies the input with the scores to get the weighted input. Finally, the 
sum of the weighted input is returned.
"""
class AttentionLayer(layers.Layer):
    def __init__(self, neurons, **kwargs):
        self.neurons = neurons
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='W', shape=(input_shape[-1], self.neurons), initializer='uniform', trainable=True)
        self.b = self.add_weight(name='b', shape=(self.neurons,), initializer='zeros', trainable=True)
        self.u = self.add_weight(name='u', shape=(self.neurons, 1), initializer='uniform', trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs, **kwargs):
        uit = tf.tanh(tf.add(tf.matmul(inputs, self.W), self.b))
        ait = tf.nn.softmax(tf.matmul(uit, self.u), axis=-1)
        weighted_input = tf.multiply(inputs, ait)
        return tf.reduce_sum(weighted_input, axis=1)

![image](workshop/attention_two_param.png)

<div class="alert alert-block alert-warning">
<b>Explore this class at home and check what each line is doing. <br>
    Also think about why? <br>
    Apply this layer to a model and see if it can improve the performance.</b>
</div>

The following section is coming from this [reference](https://colab.research.google.com/github/whitead/dmol-book/blob/master/dl/attention.ipynb#scrollTo=tXLxMdk4R9Yu)

<div class="alert alert-block alert-info">
<b>Example 2:</b> <br>
    - We have the following sentence: The sleepy child reads a book <br>
    - The goal is to see what parts of the sentence the query (for instance "book") should be influenced by. <br>
    - Let's embed each word into a vector with length 3 <br>
    - consider the query as the word "book"
    - consider the values to be the sentiment of the word. Is it a positive word ("happy") or a negative word ("angry")
</div>



| Index| Embedding| Word|
|:-----|:--------:|----:|
| 0    |  0,0,0   | The |
| 1    |  2,0,1   | Sleepy |
| 2    |  1,-1,-2   | Child |
| 3    |  2,3,1   | Reads |
| 4    |  -2,0,0   | A |
| 5    |  0,2,1   | Book |

Keys: $(6, 3)$
\begin{equation}
\mathbf{K} = \left[
\begin{array}{lccccr}
0 & 2 & 1 & 2 & -2 & 0\\
0 & 0 & -1 & 3 & 0 & 2\\
0 & 1 & -2 & 1 & 0 & 1\\
\end{array}\right]
\end{equation}
Values are the sentiment of each word: <br>
Values: $(6, 1)$
\begin{equation}
\mathbf{V} = \left[ 0, -0.2, 0.3, 0.4, 0, 0.1\right]
\end{equation}

Query: $(3,)$
\begin{equation}
\vec{q} = \left[0, 2, 1\right]
\end{equation}

In [37]:
import numpy as np

In [38]:
i_query = np.random.normal(size=(3,))
i_keys = np.random.normal(size=(6, 3))
i_values = np.random.normal(size=(6, 1))


<div class="alert alert-block alert-warning">
<b>@: dot product</b>
</div>

<div class="alert alert-block alert-success">
<b> General attenion:</b>
</div>
\begin{equation}
    \vec{b} = \mathrm{softmax}\left(\frac{1}{\sqrt{d}}\vec{q}\cdot \mathbf{K}\right)
\end{equation}

In [46]:
def softmax(x, axis=None):
    return np.exp(x) / np.sum(np.exp(x), axis=axis)


def tensor_dot(q, k):
    b = softmax((k @ q) / np.sqrt(q.shape[0]))
    return b

def attention_layer(q, k, v):
    b = tensor_dot(q, k)
    print("tensor dot result:", b)
    return b @ v
i_values = np.random.normal(size=(6, 1))
print(attention_layer(i_query, i_keys, i_values))

i_values = np.random.normal(size=(6, 2))
print(attention_layer(i_query, i_keys, i_values))

i_values = np.random.normal(size=(6, 3))
print(attention_layer(i_query, i_keys, i_values))

tensor dot result: [0.09875931 0.27359947 0.10417204 0.07964842 0.22204164 0.22177911]
[-0.09801242]


<div class="alert alert-block alert-success">
<b> Self attention:</b> when the query, values and keys are equal (We only use the keys)
</div>

In [47]:
def batched_tensor_dot(q, k):
    # a will be batch x seq x feature dim
    # which is N x N x 4
    # batched dot product in einstein notation
    a = np.einsum("ij,kj->ik", q, k) / np.sqrt(q.shape[0])
    # now we softmax over sequence
    b = softmax(a, axis=1)
    return b


def self_attention(x):
    b = batched_tensor_dot(x, x)
    return b @ x


attention_result = self_attention(i_keys)
print(attention_result.shape)
attention_result

(6, 3)


array([[ 0.37628253, -0.03922664, -0.34993863],
       [ 0.48168399,  0.91520147, -1.46493883],
       [ 0.7323018 ,  1.00371662,  0.47200163],
       [ 0.80631572, -0.36509892, -0.64357495],
       [-0.30987736,  0.33676712, -0.16211789],
       [-0.23652951,  0.2688794 , -0.25718525]])

<div class="alert alert-block alert-success">
<b> Adding Trainable Parameters:</b>
</div>

In [41]:
# weights should be input feature_dim -> desired output feature_dim
w_q = np.random.normal(size=(3, 3))
w_k = np.random.normal(size=(3, 3))
w_v = np.random.normal(size=(3, 2))


def trainable_self_attention(x, w_q, w_k, w_v):
    q = x @ w_q
    k = x @ w_k
    v = x @ w_v
    b = batched_tensor_dot(q, k)
    return b @ v


trainable_self_attention(i_keys, w_q, w_k, w_v)

array([[ 0.74485375,  1.61535542],
       [ 0.16417471, -1.22207086],
       [ 4.72068071, -3.76600936],
       [-0.40738699,  0.40194374],
       [ 1.96006861,  4.9651016 ],
       [ 0.57183174, -0.63849133]])

<div class="alert alert-block alert-success">
<b> Multi-head:</b> Use an attetnion mechanism multiple times and then concatinate the result
</div>

In [42]:
w_q_h1 = np.random.normal(size=(3, 3))
w_k_h1 = np.random.normal(size=(3, 3))
w_v_h1 = np.random.normal(size=(3, 2))
w_q_h2 = np.random.normal(size=(3, 3))
w_k_h2 = np.random.normal(size=(3, 3))
w_v_h2 = np.random.normal(size=(3, 2))
w_h = np.random.normal(size=2)


def multihead_attention(x, w_q_h1, w_k_h1, w_v_h1, w_q_h2, w_k_h2, w_v_h2):
    h1_out = trainable_self_attention(x, w_q_h1, w_k_h1, w_v_h1)
    h2_out = trainable_self_attention(x, w_q_h2, w_k_h2, w_v_h2)
    # join along last axis so we can use dot.
    all_h = np.stack((h1_out, h2_out), -1)
    return all_h @ w_h


multihead_attention(i_keys, w_q_h1, w_k_h1, w_v_h1, w_q_h2, w_k_h2, w_v_h2)

array([[-8.42021125e-03, -5.85620417e-02],
       [ 5.48249739e-02,  1.42116146e+00],
       [ 1.03971379e+00,  2.32066883e+01],
       [-1.00868714e-01,  4.20652577e-01],
       [-1.47464783e-01, -3.74156949e-01],
       [-1.98759090e-03,  7.04947641e-02]])

<div class="alert alert-block alert-info">
<b>Demo: IMDB sentiment classification.</b> <br>
    - Import the imdb dataset that contains classified reviews of the viewers of the movie. 
</div>

[More info about the dataset](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb)


In [23]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import numpy as np

N_UNIQUE_WORDS = 10000
MAXLEN = 200
(x_train, y_train),(x_test, y_test) = imdb.load_data(num_words=N_UNIQUE_WORDS)
x_train = sequence.pad_sequences(x_train, maxlen=MAXLEN)
x_test = sequence.pad_sequences(x_test, maxlen=MAXLEN)
y_train = np.array(y_train)
y_test = np.array(y_test) 

In [35]:
# Benchmark (no attention)
model = Sequential()
model.add(Embedding(N_UNIQUE_WORDS, 128, input_length=MAXLEN))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.summary()

history = model.fit(x=x_train, y=y_train, batch_size=32, epochs=5)
model.evaluate(x_test, y_test)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 200, 128)          1280000   
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 128)               98816     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 129       
Total params: 1,378,945
Trainable params: 1,378,945
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
782/782 [==============================] - 77s 93ms/step - loss: 0.3959 - accuracy: 0.8184
Epoch 2/5
782/782 [==============================] - 73s 93ms/step - loss: 0.2320 - accuracy: 0.9100
Epoch 3/5
782/782 [==============================] - 73s 94ms/step - loss: 0.1718 - accuracy: 0.9357
Epoch 4/5
782/782 [==============================] - 73s 94ms/step - l

[0.41733819246292114, 0.8570399880409241]

In [34]:
# Model with Attention
class Attention(Layer): 
    def __init__(self, num_neurons=1):    
        self.num_neurons = num_neurons
        super(Attention, self).__init__()
        
    def build(self, input_shape):
        self.num_dim_perword = input_shape[-1]
        self.words_pertweet = input_shape[-2]

        self.W = self.add_weight(
            name="att_weight",
            shape=(self.num_dim_perword, self.num_neurons),
            initializer='normal')

        self.b = self.add_weight(
            name="att_bias",
            shape=(self.words_pertweet, self.num_neurons),
            initializer='zero')
        super(Attention, self).build(input_shape)
        
    def call(self, x):
        e = K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        a = K.softmax(e)
        a = K.expand_dims(a,axis=-1)
        output = x*a
        return K.sum(output, axis=1)
    
model2 = Sequential()
model2.add(Embedding(N_UNIQUE_WORDS, 128, input_length=MAXLEN))
model2.add(Bidirectional(LSTM(64, return_sequences=True)))
model2.add(Attention()) # receive 3D and output 3D
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model2.summary()

history = model2.fit(x=x_train, y=y_train, batch_size=32, epochs=5)
model2.evaluate(x_test, y_test)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 200, 128)          1280000   
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 200, 128)          98816     
_________________________________________________________________
attention_8 (Attention)      (None, 128)               328       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 1,379,273
Trainable params: 1,379,273
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
782/782 [==============================] - 77s 93ms/step - loss: 0.3758 - accuracy: 0.8259
Epoch 2/5
782/782 [==============================] - 74s 94ms/step - loss: 0.2212 - accuracy: 0.9119
Epoch 3/5
782/782 [====================

[0.4720596373081207, 0.8588399887084961]